# Notebook 01: Data Fabric & Graph Construction

## From Tabular to Graphs: Representing Nuclear Topology

**Learning Objective:** Understand how to represent the Chart of Nuclides as a graph for GNN training using real EXFOR data.

### Why Graphs?

Nuclear data has inherent structure:
- **Isotopes** are related by reactions: U-235 + n → U-236 (capture)
- **Neighborhoods matter**: Similar Z/A have similar physics
- **Topology encodes physics**: Chart of Nuclides = Graph!

This structure is **invisible** to classical ML but **natural** for GNNs.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

from nucml_next.data import NucmlDataset
from nucml_next.utils import ChartOfNuclides

# Verify EXFOR data exists
exfor_path = Path('../data/exfor_processed.parquet')
if not exfor_path.exists():
    raise FileNotFoundError(
        f"EXFOR data not found at {exfor_path}\n"
        "Please run: python scripts/ingest_exfor.py --exfor-root <path> --output data/exfor_processed.parquet"
    )

print("✓ Imports successful")
print("✓ EXFOR data found")

### Step 1: Build the Nuclear Graph

In [ ]:
# Load real EXFOR data in GRAPH mode
dataset = NucmlDataset(
    data_path='../data/exfor_processed.parquet',
    mode='graph'
)

# Get global graph representation
graph = dataset.graph_builder.build_global_graph()

print(f"Graph Statistics:")
print(f"  Nodes (isotopes): {graph.num_nodes}")
print(f"  Edges (reactions): {graph.num_edges}")
print(f"  Node features: {graph.x.shape[1]} (includes AME2020 enrichment)")
print(f"  Edge features: {graph.edge_attr.shape[1]}")

### Step 2: Visualize the Chart of Nuclides

In [ ]:
# Plot Chart of Nuclides
chart = ChartOfNuclides()
fig, ax = chart.plot_chart(
    dataset.df[['Z', 'N']].drop_duplicates(),
    highlight_isotopes=[(92, 235), (92, 238), (94, 239)]
)
plt.show()

print("\n🔍 Observation: The chart shows nuclear topology!")
print("   • Stability valley (neutron-rich region)")
print("   • Magic numbers (peaks in stability)")
print("   • GNNs can learn this structure!")

### 🎓 Key Takeaway

> The Chart of Nuclides is a **graph**, not a table!
>
> GNNs can learn isotope relationships from real EXFOR data that classical ML cannot.
>
> With **8D node features** (including AME2020 mass excess and binding energy), 
> the graph encodes rich nuclear physics for the GNN to learn from.

Continue to `02_GNN_Transformer_Training.ipynb` →